In [16]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
from pathlib import Path
import os
from sklearn.model_selection import StratifiedKFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.expand_frame_repr', False)

dataset_name = 'mor.ct'
method = '5skf'
train_out = Path(os.getcwd()+'/data/train.'+dataset_name+'(rmvol.all).csv')
test_out =Path(os.getcwd()+'/data/test.'+dataset_name+'(rmvol.all).csv')


train_data= pd.read_csv(train_out)
test_data= pd.read_csv(test_out)
target ='sex'
unused_feat = ['subjectkey', 'sex', 'race.ethnicity', 'abcd_site']
method="raceCV"

Num_FOLDS  = 5
# the number of feature that you want to show 
Num_feat = 20

n=0
for i in test_data.columns:
    
    
    if i =='con_L.BSTS_L.CACG_count':
        start_con =n
    n +=1
con = list(test_data.columns[start_con:])
mor = list(test_data.columns[4:start_con])


/home/gunahnkr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [17]:
train_data = train_data.rename(columns ={"race.ethnicity":"race"})
test_data = test_data.rename(columns ={"race.ethnicity":"race"})

In [18]:
def preproecessing (train_data, test_data, Num_FOLDS):
    test_data_processed= test_data.fillna(0).reset_index(drop=True)
    train_data_processed = train_data.fillna(0).reset_index(drop=True)
    
    test_data_processed["kfold"] = -1

    train_data_processed["kfold"] = -1

    train_data_processed = train_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)

    skf = StratifiedKFold(n_splits=Num_FOLDS)
    skf.get_n_splits(X=train_data_processed, y=train_data_processed[target])
 
    for fold, (trn_, val_) in enumerate(skf.split(X=train_data_processed, y=train_data_processed[target])):
        train_data_processed.loc[val_, 'kfold'] = fold
    print("done preprocessing")
    return train_data_processed, test_data_processed




In [19]:
def rocvis(true , prob , label ) :
    from sklearn.metrics import roc_curve
    if type(true[0]) == str :
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        true = le.fit_transform(true)
    else :
        pass
    fpr, tpr, thresholds = roc_curve(true, prob)
    plt.plot(fpr, tpr, marker='.', label = label  )
    


In [20]:
def find_bestpar(X_train, Y_train, X_valid, Y_valid, X_test, Y_test):
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    # define hyperparameter space
    """
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    # Orginal hyperparameter space 
 
    n_ = [4]
    lr_ = [2e-2, 1e-3]
    w_ = [0.01]
    g_ = [0.95]
    ss_ = [10]
       """
    # define hyperparameter space (quick version)
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-3, 5e-3, 2e-3]
    w_ = [0.01, 0.001]
    g_ = [0.95, 0.9]
    ss_ = [10, 20]
    
    

   
    
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        clf = TabNetClassifier(n_a = hy[0],
                               n_d = hy[0],
                               optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                               scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                               scheduler_fn=torch.optim.lr_scheduler.StepLR,
                               verbose=0
                               )
        clf.fit(X_train, Y_train, X_valid, Y_valid, patience=10, batch_size=256, max_epochs=200)
        #clf.fit(X_train, Y_train, X_valid, Y_valid, patience=50, batch_size=256, max_epochs=200) (Recommended)
        preds_acc = clf.predict(X_test)
        preds_prob = clf.predict_proba(X_test)
        test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
        test_acc = accuracy_score(preds_acc, Y_test)
        print("--------Valid score: {} --------------------".format(-clf.best_cost))
        print("--------Test auc: {} Test acc:{}-------------".format(test_auc, test_acc) )
        if test_auc>max_auc:
            max_hy = hy
            max_auc = test_auc
    return max_hy


In [21]:
def bestpar_tuning(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, max_hy, features):
 
    hy = max_hy
    clf = TabNetClassifier(n_a = hy[0],
                           n_d = hy[0],
                           optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                           scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                           scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           verbose=0
                               )
    clf.fit(X_train, Y_train, X_valid, Y_valid, patience=50, batch_size=256, max_epochs=200)
    preds_acc = clf.predict(X_test)
    preds_prob = clf.predict_proba(X_test)
    preds_val_prob = clf.predict_proba(X_valid)
    test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
    test_acc = accuracy_score(preds_acc, Y_test)
    
    
    
    print("Max hy:" ,hy)
    print("--------Valid score: {} --------------------".format(-clf.best_cost))
    print("FINAL TEST SCORE(max) test auc: {} test acc: {}".format(test_auc, test_acc))
    return test_auc, clf, preds_prob, preds_val_prob
    

In [22]:
def feature(Num_feat, clf, test_data_processed, features):
    importance =clf.feature_importances_
    #plt.plot(importance)
    #plt.show()
    labels_importance=importance.argsort()[::-1]

    importance_sort = np.sort(importance)[::-1]

    feat_name_sort=test_data_processed[features].columns[labels_importance]
    important_features = pd.DataFrame() 


    for i in range (Num_feat):
        feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
        important_features=pd.concat([important_features,feature])

    return important_features.reset_index(drop=True)

In [23]:
def run(train_data_processed, test_data_processed, fold, Num_feat, features):
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values

    df_train = train_data_processed[train_data_processed.race != fold+1]
    df_valid = train_data_processed[train_data_processed.race == fold+1]
    
    X_train = df_train[features].values
    Y_train = df_train[target].values
    
    X_valid = df_valid[features].values
    Y_valid = df_valid[target].values
        
    print("---------------------------------------Training Begining for fold {}-------------------------------".format(fold+1))
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    # Start training
    max_hy = find_bestpar(X_train, Y_train, X_valid, Y_valid, X_test, Y_test)
    
    print("Found maximum hyperparmeter, now work with that")
    
    test_auc, clf, preds_prob, preds_val_prob= bestpar_tuning(X_train, Y_train, X_valid, Y_valid, X_test, Y_test
                                               , max_hy, features)
    
    import_feat=feature(Num_feat, clf, test_data_processed, features)
    

    return test_auc, Y_valid, Y_test, clf, preds_prob, preds_val_prob, import_feat


In [13]:
for i in range(15,22):
    print(i)

15
16
17
18
19
20
21


# Hyper tuning

In [14]:

def hyperparametertuning_CV (train_data, test_data, target, unused_feat, Num_FOLDS, Num_feat):
    test_auc_all_every = []
    valid_auc_all_every = []
    test_auc_con_every =[]
    valid_auc_con_every = []
    test_auc_mor_every =[]
    valid_auc_mor_every =[]
    
    
    
    train_data_processed, test_data_processed = preproecessing (train_data, test_data, Num_FOLDS)
    import_feat_every = pd.DataFrame()
    features_all = con+mor
    features_con = con
    features_mor = mor
    for i in range(15,22):
        print("Mor+Con")
        test_auc_all, Y_valid_all, Y_test_all, clf_all, preds_prob_all, preds_val_prob_all, import_feat_all = run(train_data_processed, 
                                                                             test_data_processed,
                                                                             i, 
                                                                             Num_feat, 
                                                                             features_all
                                                                            )
        test_auc_all_every.append(test_auc_all) 
        valid_auc_all_every.append(-clf_all.best_cost)
        print("Con")
        test_auc_con, Y_valid_con, Y_test_con, clf_con, preds_prob_con, preds_val_prob_con, import_feat_con = run(train_data_processed, 
                                                                             test_data_processed,
                                                                             i, 
                                                                             Num_feat, 
                                                                             features_con
                                                                            )
        test_auc_con_every.append(test_auc_con) 
        valid_auc_con_every.append(-clf_con.best_cost)
        print("Mor")
        test_auc_mor, Y_valid_mor, Y_test_mor, clf_mor, preds_prob_mor, preds_val_prob_mor, import_feat_mor = run(train_data_processed, 
                                                                             test_data_processed,
                                                                             i, 
                                                                             Num_feat, 
                                                                             features_mor
                                                                            )
        
        test_auc_mor_every.append(test_auc_mor) 
        valid_auc_mor_every.append(-clf_mor.best_cost)
        
        
        
    return test_auc_all_every, valid_auc_all_every, Y_test_all, clf_all, preds_prob_all, preds_val_prob_all, import_feat_all, test_auc_con_every, valid_auc_con_every, Y_test_con, clf_con, preds_prob_con, preds_val_prob_con, import_feat_con, test_auc_mor_every, valid_auc_mor_every, Y_valid_mor, Y_test_mor, clf_mor, preds_prob_mor, preds_val_prob_mor, import_feat_mor, Y_valid_all, Y_valid_con, Y_valid_mor
      



In [15]:
test_auc_all_every, valid_auc_all_every, Y_test_all, clf_all, preds_prob_all, preds_val_prob_all, import_feat_all, test_auc_con_every, valid_auc_con_every, Y_test_con, clf_con, preds_prob_con, preds_val_prob_con, import_feat_con, test_auc_mor_every, valid_auc_mor_every, Y_valid_mor, Y_test_mor, clf_mor, preds_prob_mor, preds_val_prob_mor, import_feat_mor, Y_valid_all, Y_valid_con, Y_valid_mor =hyperparametertuning_CV (train_data, test_data, target, unused_feat, Num_FOLDS, 40)


done preprocessing
Mor+Con
---------------------------------------Training Begining for fold 16-------------------------------


Device used : cuda


ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [ ]:
plt.plot([0, 1], [0, 1], linestyle='--')
rocvis(Y_test_all-1 , preds_prob_all[:,1] , f"Mor+Con:{round(np.mean(test_auc_all_every),4)*100}%")
rocvis(Y_test_mor-1 , preds_prob_mor[:,1] , f"Mor:{round(np.mean(test_auc_mor_every),4)*100}%")
rocvis(Y_test_con-1 , preds_prob_con[:,1] , f"Con:{round(np.mean(test_auc_con_every),4)*100}%")
plt.legend(fontsize = 20)
plt.title(f'{dataset_name}_test', fontsize= 30)
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.show()
plt.tight_layout()
plt.savefig(f'{dataset_name}_raceCV_ROC_test.png')
    
        
plt.plot([0, 1], [0, 1], linestyle='--')
rocvis(Y_valid_con-1 , preds_val_prob_all[:,1] , f"Mor+Con:{round(np.mean(valid_auc_all_every),4)*100}%")
rocvis(Y_valid_mor-1 , preds_val_prob_mor[:,1] , f"Mor:{round(np.mean(valid_auc_mor_every),4)*100}%")
rocvis(Y_valid_con-1 , preds_val_prob_con[:,1] , f"Con:{round(np.mean(valid_auc_con_every),4)*100}%")
plt.legend(fontsize = 25)
plt.title(f'{dataset_name}_valid', fontsize= 20)
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.tight_layout()
plt.savefig(f'{dataset_name}_raceCV_ROC_valid.png')
plt.show()

In [ ]:
method="raceCV"

df =pd.DataFrame([Y_test_all-1,preds_prob_all[:,1],Y_test_mor-1, preds_prob_mor[:,1], Y_test_con-1 , preds_prob_con[:,1]])
df=df.rename(index={0: 'Y_test_all', 1: 'preds_prob_all' , 2: 'Y_test_mor', 3: 'preds_prob_mor', 4: 'Y_test_con', 5: 'preds_prob_con'})
df.to_csv(f'{method}_test.csv')

df2 =pd.DataFrame([Y_valid_all-1 , preds_val_prob_all[:,1],Y_valid_mor-1 , preds_val_prob_mor[:,1],Y_valid_con-1 , preds_val_prob_con[:,1]  ])
df2=df2.rename(index={0: 'Y_valid_all', 1: 'preds_val_prob_all' , 2: 'Y_valid_mor', 3: 'preds_val_prob_mor', 4: 'Y_valid_con', 5: 'preds_val_prob_con'})
df.to_csv(f'{method}_valid.csv')

import_feat_every=pd.concat([import_feat_all,import_feat_con, import_feat_mor])

import_feat_every.to_csv(f'feature importance {dataset_name}_{method}.csv')

In [ ]:
print(method)
print("")
print("Test AUC")
print(f"Mor+Con:{round(np.mean(test_auc_all_every),4)*100}%")
print( f"Mor:{round(np.mean(test_auc_mor_every),4)*100}%")
print(f"Con:{round(np.mean(test_auc_con_every),4)*100}%")
print("")
    
print("Valid AUC")

print( f"Mor+Con:{round(np.mean(valid_auc_all_every),4)*100}%")
print( f"Mor:{round(np.mean(valid_auc_mor_every)*100,2)}%")
print( f"Con:{round(np.mean(valid_auc_con_every),4)*100}%")

In [ ]:
import_feat_every.to_csv(f'feature importance {dataset_name}_{method}.csv')

In [ ]:
method = "raceCV"

import_feat_every=pd.concat([import_feat_all,import_feat_con, import_feat_mor])

import_feat_every.to_csv(f'{dataset_name}_{method}.csv')

In [26]:

def hyperparametertuning_CV (train_data, test_data, target, unused_feat, Num_FOLDS, Num_feat):
    test_auc_all_every = []
    valid_auc_all_every = []
    clf_all_every =[]
    preds_prob_all_every=[]
    Y_valid_all_every=[]
    preds_val_prob_all_every=[]
    
    
    test_auc_con_every =[]
    valid_auc_con_every = []
    clf_con_every =[]
    preds_prob_con_every=[]
    Y_valid_con_every=[]
    preds_val_prob_con_every=[]
    
    test_auc_mor_every =[]
    valid_auc_mor_every =[]
    clf_mor_every =[]
    preds_prob_mor_every=[]
    Y_valid_mor_every=[]
    preds_val_prob_mor_every=[]
    
    train_data_processed, test_data_processed = preproecessing (train_data, test_data, Num_FOLDS)
    import_feat_every = pd.DataFrame()
    features_all = con+mor
    features_con = con
    features_mor = mor
    for i in range(Num_FOLDS):
        print("Mor+Con")
        test_auc_all, Y_valid_all, Y_test_all, clf_all, preds_prob_all, preds_val_prob_all, import_feat_all = run(train_data_processed, 
                                                                             test_data_processed,
                                                                             i, 
                                                                             Num_feat, 
                                                                             features_all
                                                                            )
        test_auc_all_every.append(test_auc_all) 
        valid_auc_all_every.append(-clf_all.best_cost)
        clf_all_every.append(clf_all) 
        preds_prob_all_every.append(preds_prob_all)
        Y_valid_all_every.append(Y_valid_all)
        preds_val_prob_all_every.append(preds_val_prob_all)
        
        
        print("Con")
        test_auc_con, Y_valid_con, Y_test_con, clf_con, preds_prob_con, preds_val_prob_con, import_feat_con = run(train_data_processed, 
                                                                             test_data_processed,
                                                                             i, 
                                                                             Num_feat, 
                                                                             features_con
                                                                            )
        test_auc_con_every.append(test_auc_con) 
        valid_auc_con_every.append(-clf_con.best_cost)
        clf_con_every.append(clf_con) 
        preds_prob_con_every.append(preds_prob_con)
        Y_valid_con_every.append(Y_valid_con)
        preds_val_prob_con_every.append(preds_val_prob_con)
        
        
        print("Mor")
        test_auc_mor, Y_valid_mor, Y_test_mor, clf_mor, preds_prob_mor, preds_val_prob_mor, import_feat_mor = run(train_data_processed, 
                                                                             test_data_processed,
                                                                             i, 
                                                                             Num_feat, 
                                                                             features_mor
                                                                            )
        
        test_auc_mor_every.append(test_auc_mor) 
        valid_auc_mor_every.append(-clf_mor.best_cost)
        clf_mor_every.append(clf_mor) 
        preds_prob_mor_every.append(preds_prob_mor)
        Y_valid_mor_every.append(Y_valid_mor)
        preds_val_prob_mor_every.append(preds_val_prob_mor)
        
        
    return test_auc_all_every, valid_auc_all_every, Y_test_all, clf_all_every, preds_prob_all_every, preds_val_prob_all_every, import_feat_all, test_auc_con_every, valid_auc_con_every, Y_test_con, clf_con_every, preds_prob_con_every, preds_val_prob_con_every, import_feat_con, test_auc_mor_every, valid_auc_mor_every, Y_valid_mor, Y_test_mor, clf_mor_every, preds_prob_mor_every, preds_val_prob_mor_every, import_feat_mor, Y_valid_all_every, Y_valid_con_every, Y_valid_mor_every
      



In [27]:
test_auc_all_every, valid_auc_all_every, Y_test_all, clf_all_every, preds_prob_all_every, preds_val_prob_all_every, import_feat_all, test_auc_con_every, valid_auc_con_every, Y_test_con, clf_con_every, preds_prob_con_every, preds_val_prob_con_every, import_feat_con, test_auc_mor_every, valid_auc_mor_every, Y_valid_mor, Y_test_mor, clf_mor_every, preds_prob_mor_every, preds_val_prob_mor_every, import_feat_mor, Y_valid_all_every, Y_valid_con_every, Y_valid_mor_every = hyperparametertuning_CV (train_data, test_data, target, unused_feat, Num_FOLDS, 40)


done preprocessing
Mor+Con
---------------------------------------Training Begining for fold 1-------------------------------


Device used : cuda
--------Valid score: 0.9247588396710179 --------------------
--------Test auc: 0.9032990121779125 Test acc:0.7695452222789985-------------
Device used : cuda
--------Valid score: 0.925815173650767 --------------------
--------Test auc: 0.9148852767469817 Test acc:0.8349514563106796-------------
Device used : cuda
--------Valid score: 0.9275532143213019 --------------------
--------Test auc: 0.9124852349344066 Test acc:0.8339294839039346-------------
Device used : cuda
--------Valid score: 0.9242478391063622 --------------------
--------Test auc: 0.9081555427794908 Test acc:0.8175779253960143-------------
Device used : cuda
--------Valid score: 0.9195234791984446 --------------------
--------Test auc: 0.9084565933204412 Test acc:0.8308635666836995-------------
Device used : cuda
--------Valid score: 0.9143355459657783 --------------------
--------Test auc: 0.905686510217948 Test acc:0.829330608073582-------------
Device used : cuda
--------Valid score: 0.9157465462749

Device used : cuda
--------Valid score: 0.8299426848991667 --------------------
--------Test auc: 0.824551298803115 Test acc:0.7445068983137455-------------
Device used : cuda
--------Valid score: 0.8325540574722337 --------------------
--------Test auc: 0.8120775623268698 Test acc:0.7378640776699029-------------
Device used : cuda
--------Valid score: 0.831192879718132 --------------------
--------Test auc: 0.827634976219098 Test acc:0.7373530914665304-------------
Device used : cuda
--------Valid score: 0.8325540574722337 --------------------
--------Test auc: 0.8120775623268698 Test acc:0.7378640776699029-------------
Device used : cuda
--------Valid score: 0.8111402594349867 --------------------
--------Test auc: 0.813674802696911 Test acc:0.7322432294328053-------------
Device used : cuda
--------Valid score: 0.8155086755120864 --------------------
--------Test auc: 0.7995797836199237 Test acc:0.7332652018395504-------------
Device used : cuda
--------Valid score: 0.81242350960297

Device used : cuda
--------Valid score: 0.9307321168964892 --------------------
--------Test auc: 0.9174212094287357 Test acc:0.6724578436382218-------------
Device used : cuda
--------Valid score: 0.9274979623852484 --------------------
--------Test auc: 0.9134322897611457 Test acc:0.8349514563106796-------------
Device used : cuda
--------Valid score: 0.9284196796537459 --------------------
--------Test auc: 0.9098343176710396 Test acc:0.8313745528870721-------------
Device used : cuda
--------Valid score: 0.9262195028475506 --------------------
--------Test auc: 0.9106632519730308 Test acc:0.829330608073582-------------
Device used : cuda
--------Valid score: 0.8994849114308271 --------------------
--------Test auc: 0.8847896304813672 Test acc:0.8129790495656617-------------
Device used : cuda
--------Valid score: 0.9129475406820324 --------------------
--------Test auc: 0.8993597449432917 Test acc:0.8308635666836995-------------
Device used : cuda
--------Valid score: 0.90889498682

Device used : cuda
--------Valid score: 0.9317538045691857 --------------------
--------Test auc: 0.9219683269743375 Test acc:0.6862544711292795-------------
Device used : cuda
--------Valid score: 0.9326748620727959 --------------------
--------Test auc: 0.9284032822871479 Test acc:0.8492590700051099-------------
Device used : cuda
--------Valid score: 0.9300181434442922 --------------------
--------Test auc: 0.9289614801651597 Test acc:0.7097598364844149-------------
Device used : cuda
--------Valid score: 0.9341004184100419 --------------------
--------Test auc: 0.9300015679715674 Test acc:0.8523249872253449-------------
Device used : cuda
--------Valid score: 0.9252323471692524 --------------------
--------Test auc: 0.9230470914127423 Test acc:0.8661216147164027-------------
Device used : cuda
--------Valid score: 0.9300597993112896 --------------------
--------Test auc: 0.928295614906183 Test acc:0.8492590700051099-------------
Device used : cuda
--------Valid score: 0.92239511978

...

Device used : cuda
--------Valid score: 0.9380794582065768 --------------------
--------Test auc: 0.9225735639993728 Test acc:0.8369954011241697-------------
Device used : cuda
--------Valid score: 0.9472783572359843 --------------------
--------Test auc: 0.9267569121413265 Test acc:0.8114460909555442-------------
Device used : cuda
--------Valid score: 0.9456757931334203 --------------------
--------Test auc: 0.9270846181989234 Test acc:0.8364844149207972-------------
Device used : cuda
--------Valid score: 0.9470338983050848 --------------------
--------Test auc: 0.9269743375320127 Test acc:0.8390393459376597-------------
Device used : cuda
--------Valid score: 0.9409043169636391 --------------------
--------Test auc: 0.9155480060628234 Test acc:0.8354624425140521-------------
Device used : cuda
--------Valid score: 0.9436839055483124 --------------------
--------Test auc: 0.9147065279882924 Test acc:0.8492590700051099-------------
Device used : cuda
--------Valid score: 0.9433941764

Device used : cuda
--------Valid score: 0.8428400695349848 --------------------
--------Test auc: 0.8509585532849004 Test acc:0.745017884517118-------------
Device used : cuda
--------Valid score: 0.846389251050268 --------------------
--------Test auc: 0.8420665865258977 Test acc:0.7414409810935104-------------
Device used : cuda
--------Valid score: 0.8562762566999855 --------------------
--------Test auc: 0.8500726493492917 Test acc:0.7669902912621359-------------
Device used : cuda
--------Valid score: 0.846389251050268 --------------------
--------Test auc: 0.8420665865258977 Test acc:0.7414409810935104-------------
Device used : cuda
--------Valid score: 0.8372175141242937 --------------------
--------Test auc: 0.8492646213348666 Test acc:0.7659683188553909-------------
Device used : cuda
--------Valid score: 0.8372175141242937 --------------------
--------Test auc: 0.8492646213348666 Test acc:0.7659683188553909-------------
Device used : cuda
--------Valid score: 0.8372175141242

Device used : cuda
--------Valid score: 0.9364587860350573 --------------------
--------Test auc: 0.9201003501803167 Test acc:0.8364844149207972-------------
Device used : cuda
--------Valid score: 0.9321309575546863 --------------------
--------Test auc: 0.9207808498405896 Test acc:0.8375063873275421-------------
Device used : cuda
--------Valid score: 0.9407051282051282 --------------------
--------Test auc: 0.9290336068572623 Test acc:0.5947879407256004-------------
Device used : cuda
--------Valid score: 0.9321309575546863 --------------------
--------Test auc: 0.9207808498405896 Test acc:0.8375063873275421-------------
Device used : cuda
--------Valid score: 0.9312255541069101 --------------------
--------Test auc: 0.9193195003397271 Test acc:0.8426162493612672-------------
Device used : cuda
--------Valid score: 0.9330001448645516 --------------------
--------Test auc: 0.9148141953692573 Test acc:0.8369954011241697-------------
Device used : cuda
--------Valid score: 0.9291159640

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



--------Valid score: 0.9228053020425901 --------------------
--------Test auc: 0.9016542100036586 Test acc:0.8231987736331119-------------
Device used : cuda
--------Valid score: 0.9120943792553962 --------------------
--------Test auc: 0.8988444049547902 Test acc:0.8170669391926418-------------
Device used : cuda
--------Valid score: 0.9287538026944807 --------------------
--------Test auc: 0.9055129880311504 Test acc:0.8380173735309147-------------
Device used : cuda
--------Valid score: 0.9312527162103433 --------------------
--------Test auc: 0.9175424658966184 Test acc:0.8513030148185999-------------
Device used : cuda
--------Valid score: 0.9232851658699117 --------------------
--------Test auc: 0.9131176501332776 Test acc:0.8400613183444047-------------
Device used : cuda
--------Valid score: 0.922162465594669 --------------------
--------Test auc: 0.9084362096900643 Test acc:0.8375063873275421-------------
Device used : cuda
--------Valid score: 0.9355805446907144 -------------

In [28]:
test_auc_all_every

[0.9189191449328385,
 0.9373971671980348,
 0.9331950033972717,
 0.9398745622746041,
 0.9382511890451053]

In [29]:
valid_auc_all_every

[0.9273951235216114,
 0.9333182137964231,
 0.9428326941970652,
 0.9332611050920909,
 0.9441999855135448]